# Export

> This module contains code for the export tab of the application

In [ ]:
#| default_exp app

In [ ]:
#| hide
from nbdev.showdoc import *

These are the relevant library calls to build out the UI and the calls to the model in order to set up the manuscript selection buttons.

In [8]:
#| export
from dash import Dash, Input, Output, callback, dcc, html
from glyptodon.annotation import *
from glyptodon.classes import *
from glyptodon.export import *
from glyptodon.information import *
from glyptodon.manuscriptFiles import *
from glyptodon.selection import *

## Instantiating the application

This first cell makes sure that the ```Dash``` application object is instantiated and able to be run. Additionally, it creates the initial layout for the application, in this case a layout based on ```Tabs```, an object which allows the user to select through the different layouts stored in a ```Tab``` object. Examples can be found in the [documentation](https://dash.plotly.com/dash-core-components/tabs).

In [9]:
#| export
app = Dash(__name__)
app.layout = html.Div(
    [
        dcc.Tabs(
            id="tabs-object",
            value="selection",
            children=[
                dcc.Tab(label="Selection", value="selection"),
                dcc.Tab(label="Information", value="information"),
                dcc.Tab(label="Annotation", value="annotation"),
                dcc.Tab(label="Export", value="export"),
            ],
        ),
        html.Div(id="current-tab"),
    ]
)

## Object calls

This next snippet of code calls and instantiates objects from ```selection```, ```information```, ```annotation```, and ```export```. In general, the id of an object returned by a method from these method calls will be a [kebab-case](https://www.freecodecamp.org/news/snake-case-vs-camel-case-vs-pascal-case-vs-kebab-case-whats-the-difference/) variant of the camelCase (e.g. the ```createManuscriptSelect()``` method call returns an object with the id ```manuscript-select```). Objects that are non-interactive or that don't need to be updated do not get ids.

In [10]:
#| export

#################
### SELECTION ###
#################
selectionKey, manuscriptSelect = createManuscriptSelect()
selectionInfo = createSelectionInfo()
finalizeSelection = createFinalizeSelection()


###################
### INFORMATION ###
###################
# the ids for objects in inputObjects are: "work", "author", "language", "country", "city", "institution"
metadata, inputObjects = createInputObjects()
centuries, centuriesSlider = createCenturiesSlider()
uploadImages, uploadeManuscripts = createUploadObjects()
informationInfo = createInformationInfo()
# the id for saveNContinue is "save-and-continue"
saveNContinue = createSaveNContinue()


##################
### ANNOTATION ###
##################
annotationTextArea = createAnnotationTextArea()
pageSelector = createPageSelector()


##############
### EXPORT ###
##############
exportInfo = createExportInfo()
exportName = createExportName()
directoryOptions = createDirectoryOptions()
exportButton = createExportButton()
exportDownload = createExportDownload()

## A callback that effects layout

The ```manuscriptSelect``` object has a **Create New Manuscript** option. If the user selects that option, we want a different layout to be available for the ```selection``` tab. This code cell sets up a [callback](https://dash.plotly.com/basic-callbacks) for the object to make sure that there is a ```bool``` to handle the conditional logic. Additionally, this callback is used to make sure that the correct manuscript is selected from the available options.

The output is a 'dummy' object created in order to have a function with no real output as per the [suggestions here](https://community.plotly.com/t/app-callback-without-an-output/5502/36).

In [11]:
newManuscript = False
selectedManuscript = selectionKey[
    "Stavronikita Monastery Greek handwritten document Collection no.53"
]


@callback(
    Output("work", "placeholder"),
    Output("author", "placeholder"),
    Output("language", "placeholder"),
    Output("country", "placeholder"),
    Output("city", "placeholder"),
    Output("institution", "placeholder"),
    Output("centuries-slider", "value"),
    Input("manuscript-select", "value"),
    allow_duplicate=True,
    prevent_initial_call=True,  # This is here to avoid tripping callbacks from the output widgets at the very beginning
    suppress_callback_exceptions=True,
)
def selectManuscript(work):
    if work == "Create New Manuscript":
        newManuscript = True
        selectedManuscript = None
        return "", "", "", "", "", "", [1, 20]
    else:
        selectedManuscript = selectionKey[work]
        work = selectedManuscript[1]["Work"]
        author = selectedManuscript[1]["Author"]
        language = selectedManuscript[1]["Language"]
        country = selectedManuscript[1]["Country"]
        city = selectedManuscript[1]["City"]
        institution = selectedManuscript[1]["Institution"]

        ### Converts the string containing centuries into list containing the centuries as integers
        # This stores the string as a list of words. There are strings with two words and four words
        centuriesAsList = selectedManuscript[1]["Centuries"].split()

        # This picks out the relevant words and strips them of everything but the integer values
        if len(centuriesAsList) == 2:
            centuriesValue = [
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
            ]
        else:
            centuriesValue = [
                int(re.sub("[^0-9]", "", centuriesAsList[0])),
                int(re.sub("[^0-9]", "", centuriesAsList[2])),
            ]
        return work, author, language, country, city, institution, centuriesValue

## Establishing layout contents

The way that tabs are changed and content is displayed is through a callback decorator run first when the application is started and then whenever a change is made in an object registered to a callback. This callback establishes the displayed tabs and what is displayed inside of them.

In [12]:
#| export
@callback(Output("current-tab", "children"), Input("tabs-object", "value"))
def renderContent(tab):
    if tab == "selection":
        return html.Div(
            [
                selectionInfo,
                html.Br(),
                manuscriptSelect,
                html.Br(),
                finalizeSelection,
            ]
        )
    elif tab == "information":
        if newManuscript:
            return html.Div(
                [
                    informationInfo,
                    html.Br(),
                    html.Div(inputObjects),
                    html.Br(),
                    html.Div(
                        [
                            uploadImages,
                            uploadManuscripts,
                        ]
                    ),
                    html.Br(),
                    saveNContinue,
                ]
            )
        else:
            return html.Div(
                [
                    informationInfo,
                    html.Br(),
                    html.Div(inputObjects),
                    html.Br(),
                    saveNContinue,
                ]
            )          
    elif tab == "annotation":
        return html.Div(
            [
                dcc.Graph(
                    id="annotation-figure",
                    config={"modeBarButtonsToAdd": ["drawline", "drawrect", "eraseshape"]},
                    style={"height":900,
                          "width":800,
                         },
                ),
                pageSelector,
                annotationTextArea,
            ]
        )
    elif tab == "export":
        return html.Div(
            [
                exportInfo,
                exportName,
                directoryOptions,
                exportButton,
                exportDownload,
            ]
        )

## Running the application

This cell contains the code that runs the application. It comes logically after all the prior code. To see the application on the local server, use http://127.0.0.1:8050/ as the website link to your locally run server.

In [ ]:
#| export
if __name__ == "__main__":
    app.run(debug=True)

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()